In [ ]:
!cd /content/drive/MyDrive/daycon-fashion

In [ ]:
!unzip /content/drive/MyDrive/daycon-fashion/Fashion_MNIST.zip

Archive:  /content/drive/MyDrive/daycon-fashion/Fashion_MNIST.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [2]:
data_dir="/content/gdrive/MyDrive/daycon-fashion/dataset"

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

label_counter=Counter(train_data['label'])
label_counter=list(sorted(label_counter.items(),key=lambda x:x[0]))

plt.plot(zip(*label_counter)[0],zip(*label_counter)[1])
plt.show()

In [16]:
label_counter

[(0, 6000),
 (1, 6000),
 (2, 6000),
 (3, 6000),
 (4, 6000),
 (5, 6000),
 (6, 6000),
 (7, 6000),
 (8, 6000),
 (9, 6000)]

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip install colab_ssh --upgrade
NGROK_TOKEN = '1aWiuL6a8VGqoiy1swWEphg0zLb_4VFYfjr1qnzHSjEyzBDwQ' # ngrok 토큰
PASSWORD = '1234' # 비밀번호 설정

from colab_ssh import launch_ssh
launch_ssh(NGROK_TOKEN, PASSWORD)

we highly recommend that update your code by following this documentation https://github.com/WassimBenzarti/colab-ssh#getting-started
Successfully running 0.tcp.ngrok.io:15476
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

	Host google_colab_ssh
		HostName 0.tcp.ngrok.io
		User root
		Port 15476
	  


In [12]:
import pandas as pd
train_data=pd.read_csv(f"{data_dir}/test.csv")
train_data.head(4)

,index,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,0,0,34,29,7,0,11,24,0,0,3,3,1,0,1,1,0,0,0,0,0,4,0,0,1,0,0,0,0,0,...,242,245,224,245,234,0,3,0,0,0,0,0,0,0,0,0,0,0,0,37,69,94,123,127,138,138,142,145,135,125,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,209,190,181,150,170,193,180,219,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,...,248,238,80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,174,233,155,0,65,235,216,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,17,0,0,0,0,0,0,0,0,12,94,68,14,0,0,0,0,0,0,0,0,0,0,0,38,106,94,89,94,68,...,0,0,0,0,0,0,87,12,26,0,0,0,0,0,0,0,46,50,29,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,63,53,31,0,0,0
3,3,0,0,0,0,0,0,0,0,0,161,212,138,150,169,164,176,202,255,183,26,0,0,0,0,0,0,0,0,0,0,0,1,0,0,129,221,255,255,215,...,255,255,254,252,255,101,196,254,252,100,0,0,0,0,0,168,232,225,0,41,172,147,148,153,155,146,137,141,143,137,137,126,140,0,133,224,222,56,0,0


In [67]:
import numpy as np

In [ ]:
testing=np.array(train_data.iloc[3])
idx,(label,image)=testing[0],testing[1],testing[2:]
len(testing)

786

In [77]:
def csv2np_img(df):
  idx=np.array(train_data['index'])
  label=np.array(train_data['label'])
  #images = shape(n_df,784)
  images=np.array(train_data.iloc[:,2:])
  return idx,label,images

In [80]:
idx,labels,images=csv2np_img(train_data)

In [76]:
images[0]

array([2, 9, 6, ..., 8, 8, 7])

In [68]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
!pip install torch
import torch

import torch.nn as nn
from importlib import import_module
import torch.nn.functional as F
#!apt-get -y install libgl1-mesa-glx
import cv2
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from time import time
import torch.utils.data as data
from torchvision import transforms  
import matplotlib.pyplot as plt
import seaborn as sns
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from torch.utils.tensorboard import SummaryWriter

from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

In [ ]:
!pip3 install torch torchvision torchaudio

In [ ]:
#make model
import torchvision.models as models
model = models.resnet50(pretrained=True)
model.conv1=torch.nn.Conv2d(1, 64, kernel_size=(2, 2), stride=(2, 2), padding=(3, 3), bias=False)
model.fc=torch.nn.Linear(in_features=2048, out_features=10, bias=True)
model.to(device)


In [78]:
#optimizer & scheduler
!pip install adamp
from adamp import AdamP

optimizer = AdamP(model.parameters(), lr=0.0001, betas=(0.9, 0.999), weight_decay=1e-2)

scheduler = ReduceLROnPlateau(optimizer, factor=0.9, patience=4) #스케줄러

In [79]:
#loss function
class CrossEntropyLoss(nn.Module):
    def __init__(self, weight=None, reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.reduction = reduction

    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            log_prob,
            target_tensor,
            weight=self.weight,
            reduction=self.reduction
        )
criterion = CrossEntropyLoss()

In [80]:
#hyper parameters
model_name="resnet50"
num_epochs=10

val_split=0.3
batch_size=32
num_workers=1

patience = 3
counter = 0

train_log_interval=200
# -- Gradient Accumulation
accumulation_steps = 2

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [81]:
class FashionDataset(Dataset):
    idx=[]
    labels=[]
    images=[]

    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        # self.mean = mean
        # self.std = std
        # self.transform = transform
        df=pd.read_csv(img_dir)
        idx=np.array(df['index'])
        labels=np.array(df['label'])
        images=np.array(df.iloc[:,2:])
        self.idx=idx
        self.labels=labels
        self.images=images
    
    def set_transform(self, transform):
        """
        transform 함수를 설정하는 함수입니다.
        """
        self.transform = transform


    def __getitem__(self, index):
        """
        데이터를 불러오는 함수입니다. 
        데이터셋 class에 데이터 정보가 저장되어 있고, index를 통해 해당 위치에 있는 데이터 정보를 불러옵니다.
        
        Args:
            index: 불러올 데이터의 인덱스값입니다.
        """
        # 이미지를 불러옵니다.
        images=self.images
        labels=self.labels
        
        # 이미지를 Augmentation 시킵니다.
        # image_transform = self.transform(image=np.array(image))['image']
#         return image_transform, multi_class_label
        return labels[index],images[index]

    def __len__(self):
        return len(self.idx)

In [82]:
#dataset, dataloader
# train dataset과 validation dataset을 사전 정의한 비율로 나눕니다.
train_file="/content/gdrive/MyDrive/daycon-fashion/dataset/train.csv"
test_file="/content/gdrive/MyDrive/daycon-fashion/dataset/test.csv"

dataset=FashionDataset(
    img_dir=train_file
)
n_val = int(len(dataset) *val_split)
n_train = len(dataset) - n_val
train_dataset, val_dataset = data.random_split(dataset, [n_train, n_val])

# 각 dataset에 augmentation 함수를 설정합니다.
# train_dataset.dataset.set_transform(transform['train'])
# val_dataset.dataset.set_transform(transform['val'])

In [83]:
# training dataloader은 데이터를 섞어주어야 합니다. (shuffle=True)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False,
    drop_last=True
)

In [84]:
#train
os.makedirs(os.path.join("/content/gdrive/MyDrive/daycon-fashion", 'results', model_name), exist_ok=True)
best_val_acc = 0
best_val_loss = np.inf
for epoch in tqdm(range(num_epochs)):
    # train loop
    model.train()
    loss_value = 0
    matches = 0
    
    for idx, train_batch in enumerate(train_loader):
        labels,inputs=train_batch
        labels=labels.to(device)
        #size 맞춰주기
        inputs = inputs.reshape(batch_size,1,28,28)/255
        inputs = inputs.to(device)
        
        outs = model(inputs)
    
        loss=criterion(outs,labels)
        loss.backward()   
        # -- Gradient Accumulation
        if (idx+1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            
        pred= torch.argmax(outs,dim=-1)

        loss_value += loss.item()
        matches += (pred == labels).sum().cpu().numpy()
        
        if (idx + 1) % train_log_interval == 0:
            train_loss = loss_value /train_log_interval
            train_acc = matches /batch_size/ train_log_interval
            print(
                f"Epoch[{epoch}/{num_epochs}]({idx + 1}/{len(train_loader)}) || "
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || "
            )
            loss_value = 0
            matches = 0

    scheduler.step(loss)

    # val loop
    with torch.no_grad():
        print("Calculating validation results...")
        model.eval()
        val_loss_items = []
        val_acc_items = []
        
        for val_batch in val_loader:
            labels,inputs = val_batch
            inputs = inputs.reshape(batch_size,1,28,28)/255
            inputs = inputs.to(device)
            
            labels=labels.to(device)

            outs = model(inputs)

            pred = torch.argmax(outs, dim=-1)
            
            loss_item=criterion(outs,labels)
        
            acc_item=(pred == labels).sum().cpu().numpy()

            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            
        val_loss = np.sum(val_loss_items) / len(val_loader)
        val_acc = np.sum(val_acc_items) / len(val_dataset)
        
        # Callback1: validation accuracy가 향상될수록 모델을 저장합니다.
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        if val_acc > best_val_acc:
            print("New best model for val accuracy! saving the model..")
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"/content/gdrive/MyDrive/daycon-fashion/results/{model_name}/{model_name}{epoch:03}_accuracy_{best_val_acc:4.2%}_loss_{best_val_loss:4.4%}.pth")
            counter = 0
        else:
            counter += 1
        # Callback2: patience 횟수 동안 성능 향상이 없을 경우 학습을 종료시킵니다.
        if counter > patience:
            print("Early Stopping...")
            break
        
        
        print(
            f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
            f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2} || "
        )










  0%|          | 0/10 [00:00<?, ?it/s]

Epoch[0/10](200/1312) || training loss 0.9588 || training accuracy 66.09% || 
Epoch[0/10](400/1312) || training loss 0.5609 || training accuracy 79.80% || 
Epoch[0/10](600/1312) || training loss 0.5117 || training accuracy 81.61% || 
Epoch[0/10](800/1312) || training loss 0.4601 || training accuracy 83.41% || 
Epoch[0/10](1000/1312) || training loss 0.4248 || training accuracy 84.25% || 
Epoch[0/10](1200/1312) || training loss 0.429 || training accuracy 84.11% || 
Calculating validation results...
New best model for val accuracy! saving the model..











 10%|█         | 1/10 [01:26<12:58, 86.54s/it]

[Val] acc : 86.30%, loss: 0.37 || best acc : 86.30%, best loss: 0.37 || 
Epoch[1/10](200/1312) || training loss 0.3418 || training accuracy 87.39% || 
Epoch[1/10](400/1312) || training loss 0.3457 || training accuracy 87.77% || 
Epoch[1/10](600/1312) || training loss 0.3537 || training accuracy 87.48% || 
Epoch[1/10](800/1312) || training loss 0.3254 || training accuracy 88.39% || 
Epoch[1/10](1000/1312) || training loss 0.3423 || training accuracy 87.47% || 
Epoch[1/10](1200/1312) || training loss 0.3318 || training accuracy 87.47% || 
Calculating validation results...
New best model for val accuracy! saving the model..











 20%|██        | 2/10 [02:54<11:36, 87.09s/it]

...

[Val] acc : 88.87%, loss: 0.32 || best acc : 89.27%, best loss:  0.3 || 
Epoch[5/10](200/1312) || training loss 0.1354 || training accuracy 94.98% || 
Epoch[5/10](400/1312) || training loss 0.1447 || training accuracy 94.69% || 
Epoch[5/10](600/1312) || training loss 0.1578 || training accuracy 94.09% || 
Epoch[5/10](800/1312) || training loss 0.1717 || training accuracy 93.55% || 
Epoch[5/10](1000/1312) || training loss 0.1713 || training accuracy 93.53% || 
Epoch[5/10](1200/1312) || training loss 0.1662 || training accuracy 93.98% || 
Calculating validation results...











 60%|██████    | 6/10 [08:47<05:50, 87.71s/it]

[Val] acc : 89.21%, loss: 0.33 || best acc : 89.27%, best loss:  0.3 || 
Epoch[6/10](200/1312) || training loss 0.1131 || training accuracy 95.70% || 
Epoch[6/10](400/1312) || training loss 0.1228 || training accuracy 95.50% || 
Epoch[6/10](600/1312) || training loss 0.1327 || training accuracy 95.20% || 
Epoch[6/10](800/1312) || training loss 0.1251 || training accuracy 95.33% || 
Epoch[6/10](1000/1312) || training loss 0.1409 || training accuracy 94.83% || 
Epoch[6/10](1200/1312) || training loss 0.1453 || training accuracy 94.88% || 
Calculating validation results...











 70%|███████   | 7/10 [10:14<04:23, 87.69s/it]

[Val] acc : 88.99%, loss: 0.34 || best acc : 89.27%, best loss:  0.3 || 
Epoch[7/10](200/1312) || training loss 0.09418 || training accuracy 96.61% || 
Epoch[7/10](400/1312) || training loss 0.09215 || training accuracy 96.78% || 
Epoch[7/10](600/1312) || training loss 0.1118 || training accuracy 95.78% || 
Epoch[7/10](800/1312) || training loss 0.1053 || training accuracy 96.03% || 
Epoch[7/10](1000/1312) || training loss 0.1195 || training accuracy 95.50% || 
Epoch[7/10](1200/1312) || training loss 0.1159 || training accuracy 95.88% || 
Calculating validation results...
Early Stopping...


In [85]:
import torch.utils.data as data
from torchvision import transforms  
import matplotlib.pyplot as plt
import seaborn as sns
import torch.optim as optim
# mean, std = (0.56019358 ,0.52410121, 0.501457), (0.23318603, 0.24300033 ,0.24567522)
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize,CenterCrop
class TestDataset(Dataset):
    def __init__(self, img_paths, transform=None):
        df=pd.read_csv(img_paths)
        idx=np.array(df['index'])
        images=np.array(df.iloc[:,1:])
        self.idx=idx
        self.images=images

    def __getitem__(self, index):
        image = self.images
        # if self.transform:
        #     image = self.transform(image)
        return self.images[index]

    def __len__(self):
        return len(self.idx)
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir='/content/gdrive/MyDrive/daycon-fashion/dataset/test.csv'

submission = pd.read_csv('/content/gdrive/MyDrive/daycon-fashion/dataset/sample_submission.csv')
# image_dir = os.path.join(test_dir, 'images')
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
# image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
# transform = transforms.Compose([
#     CenterCrop((380,380)),
#     Resize((300, 300), Image.BILINEAR),
#     ToTensor(),
#     Normalize(mean=mean, std=std),
    
# ])
transform=None
dataset = TestDataset(test_dir, transform)

loader = DataLoader(
    dataset
)
# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)

model_root = f'/content/gdrive/MyDrive/daycon-fashion/results/resnet50/resnet50003_accuracy_89.27%_loss_30.0600%.pth'
model = models.resnet50(pretrained=True)
model.conv1=torch.nn.Conv2d(1, 64, kernel_size=(2, 2), stride=(2, 2), padding=(3, 3), bias=False)
model.fc=torch.nn.Linear(in_features=2048, out_features=10, bias=True)
model.to(device)
# from torchvision.models import resnet50
# model = resnet50(pretrained=True)
model.load_state_dict(torch.load(model_root))
model.eval()
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for i,images in tqdm(enumerate(loader)):
    with torch.no_grad():
        images = images.to(device)
        images=images.reshape(1,1,28,28)/255
        pred = model(images)
        
        pred = torch.argmax(pred, dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['label'] = all_predictions
print(submission)

# # 제출할 파일을 저장합니다.
# submission.to_csv(os.path.join(test_dir, 'try6whole_effi005_accuracy_98.50%_loss_4.7256%.csv'), index=False)
# print('test inference is done!')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.









1069it [00:06, 169.55it/s]









1088it [00:06, 173.10it/s]









1106it [00:06, 167.44it/s]









1124it [00:06, 170.66it/s]









1142it [00:06, 172.01it/s]









1161it [00:06, 175.39it/s]









1179it [00:06, 171.87it/s]









1197it [00:06, 174.08it/s]









1215it [00:06, 171.00it/s]









1233it [00:07, 165.87it/s]









1251it [00:07, 167.81it/s]









1270it [00:07, 171.65it/s]









1288it [00:07, 168.81it/s]









1306it [00:07, 171.84it/s]









1325it [00:07, 175.83it/s]









1343it [00:07, 175.61it/s]









1361it [00:07, 175.04it/s]









1379it [00:07, 172.34it/s]









1397it [00:08, 171.21it/s]









1416it [00:08, 174.68it/s]









1434it [00:08, 174.96it/s]









1452it [00:08, 174.52it/s]









1470it [00:08, 175.96it/s]









1489it [00:08, 177.38it/s]









1508it [00:08, 178.68it/s]









1527it [00:08, 179.68it/s]









1545it [00:08, 177.

      index  label
0         0      0
1         1      1
2         2      2
3         3      2
4         4      3
...     ...    ...
9995   9995      0
9996   9996      6
9997   9997      8
9998   9998      8
9999   9999      1

[10000 rows x 2 columns]


In [86]:
pred_dir="/content/gdrive/MyDrive/daycon-fashion/predictions"
submission.to_csv(f'{pred_dir}/output2.csv', index=False)
print('test inference is done!')

test inference is done!


In [87]:
len(submission)

10000

AttributeError: ignored

In [ ]:
resnet18.to(device)
resnet18.eval()
resnet18(torch.tensor(test_image1).to(torch.float32).to(device))

In [48]:
resnet18

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  